# PINN Advection-Diffusion - Robustness
Autor: Taco de Wolff\
Fecha: 25 mayo 2021

Reconstruir la ecuación de advección-difusión usando Physics-Informed Neural Networks (PINNs). La ecuación de advección-difusión, que tiene de entrada la velocidad $U$ y el coeficiente de difusión $D$, y de salida la temperatura $T$, la PDE es:

$ \frac{\partial T}{\partial t} = D \nabla^2 T - U \cdot \nabla T$

nos da

$f = \frac{\partial T}{\partial t} - D\left(\frac{\partial^2 T}{\partial x^2} + \frac{\partial^2 T}{\partial y^2}\right) + u\frac{\partial T}{\partial x} + v\frac{\partial T}{\partial y} = 0$.

In [1]:
import wandb
import numpy as np
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import grad

wandb.login();

wandb: Currently logged in as: oceania (use `wandb login --relogin` to force relogin)


In [2]:
x_data = np.loadtxt("../Data/AdvDif/X_star.txt") # N x 2  
t_data = np.loadtxt("../Data/AdvDif/T_star.txt") # T
T_data = np.loadtxt("../Data/AdvDif/Temp.txt")   # N x T
xy = np.tile(x_data.T, (t_data.shape[0],1)) # TN x 2
t = np.repeat(t_data.reshape(-1,1), x_data.shape[0], axis=0) # TN x 1
X = np.concatenate([xy,t], axis=1) # TN x 3
Y = T_data.T.reshape(-1,1) # TN x 1

print("x.shape:", x_data.shape)
print("t.shape:", t_data.shape)
print("T.shape:", T_data.shape)
print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("X: %s ± %s" % (X.mean(axis=0), X.std(axis=0)))
print("Y: %s ± %s" % (Y.mean(axis=0), Y.std(axis=0)))

# normalize data
Y_mean = np.mean(Y, axis=0)
Y -= Y_mean
Y_std = np.std(Y, axis=0)
Y /= Y_std

x.shape: (2, 819)
t.shape: (500,)
u.shape: (819, 500)
v.shape: (819, 500)
T.shape: (819, 500)
samples: 409500
X: [0.49457726 0.74749999 0.5       ] ± [0.29588245 0.44382881 0.28925306]
Y: [-3.14833308e-04 -4.30505223e-04  2.92408708e+02] ± [ 0.93504805  1.05144072 14.06447006]


In [3]:
class PINN_AD(nn.Module):
    def __init__(self, layer_width=40, layer_depth=3,
                 activation_function='tanh', initializer='none',
                 Y_mean=None, Y_std=None): 
        super().__init__()
        
        input_width = 3
        output_width = 1
        
        self.Y_mean = Y_mean
        self.Y_std = Y_std
        self.D = 0.02
        self.u = np.cos(22.5*np.pi/180.0)
        self.v = np.sin(22.5*np.pi/180.0)
        
        sizes = [input_width] + [layer_width]*layer_depth + [output_width]
        self.net = nn.Sequential(
            *[self.block(dim_in, dim_out, activation_function)
            for dim_in, dim_out in zip(sizes[:-1], sizes[1:-1])],
            nn.Linear(sizes[-2], sizes[-1]) # output layer is regular linear transformation
        )
        
        if initializer == 'xavier':
            def init_weights(m):
                if type(m) == nn.Linear:
                    torch.nn.init.xavier_uniform_(m.weight)
            self.net.apply(init_weights)
                
    def forward(self, x):
        return self.net.forward(x)
    
    def block(self, dim_in, dim_out, activation_function):
        activation_functions = nn.ModuleDict([
            ['lrelu', nn.LeakyReLU()],
            ['relu', nn.ReLU()],
            ['tanh', nn.Tanh()],
            ['sigmoid', nn.Sigmoid()],
            ['softplus', nn.Softplus()],
            ['softsign', nn.Softsign()],
            ['tanhshrink', nn.Tanhshrink()],
            ['celu', nn.CELU()],
            ['gelu', nn.GELU()],
            ['elu', nn.ELU()],
            ['selu', nn.SELU()],
            ['logsigmoid', nn.LogSigmoid()]
        ])
        return nn.Sequential(
            nn.Linear(dim_in, dim_out),
            activation_functions[activation_function],
        )
        
    def f(self, x, y, t, T):
        if Y_mean is not None and Y_std is not None:
            T = T*self.Y_std[0] + self.Y_mean[0]
        
        T_x  = grad(T,   x, create_graph=True, grad_outputs=torch.ones_like(T))[0]
        T_xx = grad(T_x, x, create_graph=True, grad_outputs=torch.ones_like(T_x))[0]
        T_y  = grad(T,   y, create_graph=True, grad_outputs=torch.ones_like(T))[0]
        T_yy = grad(T_y, y, create_graph=True, grad_outputs=torch.ones_like(T_y))[0]
        T_t  = grad(T,   t, create_graph=True, grad_outputs=torch.ones_like(T))[0]
        return T_t - self.D*(T_xx + T_yy) + self.u*T_x + sel.v*T_y
    
    def loss(self, Xu, Yu, Xf=None):
        losses = []
        losses.append(F.mse_loss(self.forward(Xu), Yu))
        
        if Xf is not None:
            Xf.requires_grad=True
            x = Xf[:,0]
            y = Xf[:,1]
            t = Xf[:,2]
            Xf = torch.stack((x,y,t),1)
            Y_hat = self.forward(Xf)
            T = Y_hat[:,0]        
            f = self.f(x, y, t, T)        
            losses.append(F.mse_loss(f, torch.zeros_like(f)))
        return losses

In [6]:
# parameters
project = 'advdif'
epochs = 20000
N = X.shape[0]

torch.manual_seed(2021)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device:", device)

rng = np.random.default_rng(2021)
seeds = rng.integers(1000, size=25)

# sweep and train
sweep_config = {
    'project': project,
    'method': 'grid',
    'parameters': {
        'alpha': {
            'values': [0.0, 0.5]
        },
        'seed': {
            'values': [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]
        },
        'Nu': {
            'values': [500]
        },
        'Nf': {
            'values': [10000]
        }
    }   
}
sweep_id = wandb.sweep(sweep_config)

# train
models = []
def model_train():
    run = wandb.init()
    config = wandb.config
    rng = np.random.default_rng(seeds[config.seed])
    name = 'advdif_a%g_nu%g_nf%g_s%d' % (config.alpha, config.Nu, config.Nf, config.seed)
    
    # data
    Nu = int(config.Nu)
    Xu_idx = rng.choice(X.shape[0], Nu, replace=False)
    Xu = X[Xu_idx,:]
    Yu = Y[Xu_idx,:]

    Nf = int(config.Nf)
    Xf_idx = rng.choice(X.shape[0], Nf, replace=False)
    Xf = X[Xf_idx,:]

    print("Xu.shape:", Xu.shape)
    print("Yu.shape:", Yu.shape)
    print("Xf.shape:", Xf.shape)

    Xu = torch.tensor(Xu, dtype=torch.float, device=device)
    Yu = torch.tensor(Yu, dtype=torch.float, device=device)
    Xf = torch.tensor(Xf, dtype=torch.float, device=device)
    Xval = torch.tensor(X, dtype=torch.float, device=device)
    Yval = torch.tensor(Y, dtype=torch.float, device=device)
        
    # model
    model = PINN_AD(layer_width=40,
                    layer_depth=3,
                    activation_function='tanh',
                    initializer='xavier',
                    Y_mean=Y_mean,
                    Y_std=Y_std
                   )
    model.to(device)    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.002)
            
    # training
    val_data_losses = np.array([])
    start_time = time.time()
    for epoch in range(epochs):
        if config.alpha != 0.0:
            losses = model.loss(Xu, Yu, Xf)
            train_data_loss = (1.0-config.alpha)*losses[0]
            phys_loss = config.alpha*losses[1]
            loss = train_data_loss + phys_loss
        else:
            losses = model.loss(Xu, Yu)
            train_data_loss = losses[0]
            phys_loss = torch.tensor(0.0)
            loss = train_data_loss        
        wandb.log({'Data loss (training)': train_data_loss.detach().item()}, step=epoch)
        wandb.log({'Physics loss': phys_loss.detach().item()}, step=epoch)
        wandb.log({'Loss (training)': loss.item()}, step=epoch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses = model.loss(Xval, Yval)
        val_data_loss = losses[0]
        val_loss = val_data_loss + phys_loss
        wandb.log({'Data loss (validation)': val_data_loss.item()}, step=epoch)
        wandb.log({'Loss (validation)': val_loss.item()}, step=epoch)
           
        val_data_losses = np.append(val_data_losses, val_data_loss.item())
        if 250 <= len(val_data_losses):
            lowest_val_data_loss = np.min(val_data_losses[-250:])
        else:
            lowest_val_data_loss = np.min(val_data_losses)
        wandb.log({'Data loss lowest (validation)': lowest_val_data_loss}, step=epoch)
        
        if epoch % 1000 == 0:            
            elapsed = time.time() - start_time
            print('Epoch: %d, Loss: %.3e, Time: %.2fs' % 
                      (epoch, val_data_loss, elapsed))
            start_time = time.time()

    torch.save(model.state_dict(), name + '.pth')
    models.append(model)

wandb.agent(sweep_id, project=project, function=model_train);

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Device: cpu


KeyboardInterrupt: 

In [ ]:
wandb.finish()